In [1]:
import pandas as pd
from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import StackingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
import plotly.express as px

In [2]:
drive.mount('/content/drive')
path = "/content/drive/MyDrive/Capstone/neiss_2022.csv"
neiss = pd.read_csv(path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Initialize the label encoder
le = LabelEncoder()

## Diagnosis - XGBoost

### Initial Model

In [4]:
X_diag = neiss.drop(['CPSC_Case_Number','Other_Race', 'Diagnosis','Other_Diagnosis', 'Body_Part_2', 'Diagnosis_2', 'Other_Diagnosis_2', 'PSU', 'Stratum', 'Narrative_1','Treatment_Date'], axis=1)
y_diag = le.fit_transform(neiss['Diagnosis'])

In [5]:
# Splitting data for `diagnosis`
X_train_diag, X_test_diag, y_train_diag, y_test_diag = train_test_split(X_diag, y_diag, test_size=0.2, random_state=42)

In [6]:
y_train_diag = le.fit_transform(y_train_diag)

In [7]:
# Create an XGBClassifier instance with GPU support
model_xg_diag = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', tree_method='gpu_hist', gpu_id=0)

In [8]:
# Fit the model on training data
model_xg_diag.fit(X_train_diag, y_train_diag)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [16:23:27] WARNING: /workspace/src/common/error_msg.cc:45: `gpu_id` is deprecated since2.0.0, use `device` instead. E.g. device=cpu/cuda/cuda:0
  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [16:23:27] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, gpu_id=0, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, ...)

In [9]:
# Make predictions
y_pred_diag = model_xg_diag.predict(X_test_diag)
y_pred_diag = le.inverse_transform(y_pred_diag)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [16:23:49] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [16:23:49] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


In [10]:
# Evaluate the model
accuracy_diag = accuracy_score(y_test_diag, y_pred_diag)
print(f'Accuracy of XGBoost Classifier: {accuracy_diag}')

Accuracy of XGBoost Classifier: 0.5992825001159752


In [11]:
feature_importances_diag = model_xg_diag.feature_importances_

In [12]:
feature_names = X_train_diag.columns

In [13]:
df_sorted = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances_diag})
df_sorted = df_sorted.sort_values(by='Importance', ascending=False)

In [14]:
fig = px.bar(df_sorted, x='Feature', y='Importance', labels={'x': 'Feature', 'y': 'Importance'},
             title="Feature Importances")
fig.show()

In [15]:
print(classification_report(y_test_diag, y_pred_diag))

              precision    recall  f1-score   support

           0       0.94      0.98      0.96       794
           1       0.50      0.23      0.32        65
           2       1.00      0.18      0.30        17
           3       0.00      0.00      0.00        22
           4       0.79      0.76      0.77       427
           5       0.46      0.30      0.36        91
           6       0.55      0.16      0.25       142
           7       0.70      0.60      0.64       481
           8       0.52      0.22      0.31      1307
           9       0.40      0.23      0.29      7022
          10       0.25      0.11      0.15       116
          11       0.50      0.25      0.34      1024
          12       0.74      0.70      0.72      1442
          13       0.57      0.68      0.62     10663
          14       0.28      0.02      0.03       896
          15       0.63      0.70      0.66     10174
          16       0.45      0.34      0.39       345
          17       0.12    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



### Feature Importance

In [ ]:
X_diag_fi = neiss.drop(['CPSC_Case_Number','Other_Race', 'Diagnosis','Other_Diagnosis', 'Body_Part_2', 'Diagnosis_2', 'Other_Diagnosis_2', 'PSU', 'Stratum', 'Narrative_1','Treatment_Date','Drug','Product_3','Alcohol','Fire_Involvement'], axis=1)
y_diag_fi = le.fit_transform(neiss['Diagnosis'])

In [ ]:
# Splitting data for `diagnosis`
X_train_diag_fi, X_test_diag_fi, y_train_diag_fi, y_test_diag_fi = train_test_split(X_diag_fi, y_diag_fi, test_size=0.2, random_state=42)

In [ ]:
# Scale the features
scaler_diag_fi = StandardScaler()
X_train_diag_fi = scaler_diag_fi.fit_transform(X_train_diag_fi)
X_test_diag_fi = scaler_diag_fi.transform(X_test_diag_fi)

In [ ]:
# Create an XGBClassifier instance with GPU support
model_xg_diag_fi = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', tree_method='gpu_hist', gpu_id=0)

In [ ]:
# Fit the model on training data
model_xg_diag_fi.fit(X_train_diag_fi, y_train_diag_fi)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [20:56:05] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, gpu_id=0, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, ...)

In [ ]:
# Make predictions
y_pred_diag_fi = model_xg_diag_fi.predict(X_test_diag_fi)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [20:56:19] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


In [ ]:
# Evaluate the model
accuracy_diag_fi = accuracy_score(y_test_diag_fi, y_pred_diag_fi)
print(f'Accuracy of XGBoost Classifier: {accuracy_diag_fi}')

Accuracy of XGBoost Classifier: 0.5879633209111012


### Best Parameters - Initial

In [ ]:
# Create an XGBClassifier instance with GPU support
model_xg_diag_bp = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', tree_method='gpu_hist', gpu_id=0)

In [ ]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 6, 9],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0],
}

In [ ]:
# Initialize GridSearchCV
grid_search_diag_initial = GridSearchCV(estimator=model_xg_diag_bp, param_grid=param_grid, cv=3, scoring='accuracy', verbose=2)


In [ ]:
grid_search_diag_initial.fit(X_train_diag, y_train_diag)

Fitting 3 folds for each of 243 candidates, totalling 729 fits


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [22:37:29] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [22:37:35] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.8; total time=   6.8s


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [22:37:35] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [22:37:40] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.8; total time=   5.5s


...

[CV] END colsample_bytree=1.0, learning_rate=0.2, max_depth=9, n_estimators=300, subsample=1.0; total time=  30.4s


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [01:45:22] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [01:45:52] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


[CV] END colsample_bytree=1.0, learning_rate=0.2, max_depth=9, n_estimators=300, subsample=1.0; total time=  30.5s


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [01:45:53] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


GridSearchCV(cv=3,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False,
                                     eval_metric='mlogloss', feature_types=None,
                                     gamma=None, gpu_id=0, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     lear...
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None, ...),
             param_grid={'colsample_bytree': [0.8, 0.9, 1.0],
                         'learning_rate': [0.01, 0.1, 0.2],
                         'max_depth': [3, 6, 9],
                         'n_estimators': [100, 200, 300],
                         'subsample': [0.8, 0.9, 1.0]},
             scoring='accuracy', verbose=2)

In [ ]:
# The best combination of parameters and their best score
best_params = grid_search_diag_initial.best_params_
best_score = grid_search_diag_initial.best_score_

In [ ]:
print(f"Best parameters: {best_params}")
print(f"Best cross-validation score: {best_score}")

Best parameters: {'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 9, 'n_estimators': 100, 'subsample': 0.9}
Best cross-validation score: 0.596569435153881


### Best Parameters - FI

In [16]:
# Best parameters found by GridSearchCV
best_params = {
    'colsample_bytree': 0.8,
    'learning_rate': 0.1,
    'max_depth': 9,
    'n_estimators': 100,
    'subsample': 0.9
}

In [17]:
# Create a new model with the best parameters
best_xgb_model = XGBClassifier(
    colsample_bytree=best_params['colsample_bytree'],
    learning_rate=best_params['learning_rate'],
    max_depth=best_params['max_depth'],
    n_estimators=best_params['n_estimators'],
    subsample=best_params['subsample'],
    tree_method='gpu_hist',
    gpu_id=0,
)

In [ ]:
best_xgb_model.fit(X_train_diag_fi, y_train_diag_fi)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [01:54:00] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=0, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=9, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, ...)

In [ ]:
y_pred_diag_fi_best = best_xgb_model.predict(X_test_diag_fi)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [01:55:15] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


In [ ]:
# Evaluate the model
accuracy_diag_fi_best = accuracy_score(y_test_diag_fi, y_pred_diag_fi_best)
print(f'Accuracy of XGBoost Classifier: {accuracy_diag_fi_best}')

Accuracy of XGBoost Classifier: 0.5974732870463437


### Stacking XGBoost and RF



In [ ]:
# Define estimators
estimators = [
    ('rf', RandomForestClassifier(n_estimators=300,
                                  max_depth=20,
                                  min_samples_leaf=1,
                                  min_samples_split=10,
                                  random_state=42)),
    ('xgb', XGBClassifier(use_label_encoder=False,
                          eval_metric='logloss',
                          colsample_bytree=best_params['colsample_bytree'],
                          learning_rate=best_params['learning_rate'],
                          max_depth=best_params['max_depth'],
                          n_estimators=best_params['n_estimators'],
                          subsample=best_params['subsample'],
                          tree_method='gpu_hist',
                          gpu_id=0 ))
]

In [ ]:
# Define stacking classifier with Logistic Regression as the final estimator
stack_model = StackingClassifier(
    estimators=estimators,
    final_estimator=LogisticRegression(),
    cv=5  # Cross-validation strategy for the blending step
)

In [ ]:
# Train the stacking classifier
stack_model.fit(X_train_diag_fi, y_train_diag_fi)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [15:44:33] WARNING: /workspace/src/common/error_msg.cc:45: `gpu_id` is deprecated since2.0.0, use `device` instead. E.g. device=cpu/cuda/cuda:0
  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [15:44:33] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [15:51:17] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [

StackingClassifier(cv=5,
                   estimators=[('rf',
                                RandomForestClassifier(max_depth=20,
                                                       min_samples_split=10,
                                                       n_estimators=300,
                                                       random_state=42)),
                               ('xgb',
                                XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=0.8, device=None,
                                              early_stopping_rounds=None,
                                              enable_categorical=False,
                                              eval_metric='logloss'...
                                              importance_type=None,
                                              interaction_constraints=None,
                                              learning_rate=0.1, max_bin=None,
                                              max_cat_threshold=None,
                                              max_cat_to_onehot=None,
                                              max_delta_step=None, max_depth=9,
                                              max_leaves=None,
                                              min_child_weight=None,
                                              missing=nan,
                                              monotone_constraints=None,
                                              multi_strategy=None,
                                              n_estimators=100, n_jobs=None,
                                              num_parallel_tree=None, ...))],
                   final_estimator=LogisticRegression())

In [ ]:
# Evaluate the model
stack_model_score = stack_model.score(X_test_diag_fi, y_test_diag_fi)
print(f'Stacking Classifier Score: {stack_model_score}')

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [15:56:19] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


Stacking Classifier Score: 0.5941486647389012


In [ ]:
from joblib import dump

In [ ]:
dump(stack_model, '/content/drive/MyDrive/Capstone/neiss_stack_model.pkl')

### Stacking XGBoost, RF, and SVC

In [ ]:
# Define base estimators
estimators = [
    ('rf', RandomForestClassifier(n_estimators=10, random_state=42)),
    ('svc', make_pipeline(StandardScaler(), SVC(random_state=42))),
    ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='logloss'))
]

In [ ]:
# Define stacking classifier with Logistic Regression as the final estimator
stack_model_2 = StackingClassifier(
    estimators=estimators,
    final_estimator=LogisticRegression(),
    cv=5  # Cross-validation strategy for the blending step
)

In [ ]:
# Train the stacking classifier
stack_model_2.fit(X_train_diag, y_train_diag)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


StackingClassifier(cv=5,
                   estimators=[('rf',
                                RandomForestClassifier(n_estimators=10,
                                                       random_state=42)),
                               ('svc',
                                Pipeline(steps=[('standardscaler',
                                                 StandardScaler()),
                                                ('svc',
                                                 SVC(random_state=42))])),
                               ('xgb',
                                XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=None,
                                              device=None,
                                              early_stopping_r...
                                              interaction_constraints=None,
                                              learning_rate=None, max_bin=None,
                                              max_cat_threshold=None,
                                              max_cat_to_onehot=None,
                                              max_delta_step=None,
                                              max_depth=None, max_leaves=None,
                                              min_child_weight=None,
                                              missing=nan,
                                              monotone_constraints=None,
                                              multi_strategy=None,
                                              n_estimators=None, n_jobs=None,
                                              num_parallel_tree=None,
                                              random_state=None, ...))],
                   final_estimator=LogisticRegression())

In [ ]:
from joblib import dump

In [ ]:
dump(stack_model_2, '/content/drive/MyDrive/Capstone/neiss_stack_model_2.pkl')

['/content/drive/MyDrive/Capstone/neiss_stack_model_2.pkl']

In [ ]:
# Evaluate the model
stack_model_score_2 = stack_model_2.score(X_test_diag, y_test_diag)
print(f'Stacking Classifier Score: {stack_model_score_2}')

Stacking Classifier Score: 0.45671032488518454


In [ ]:
# Evaluate the model
stack_model_score = stack_model.score(X_test_diag, y_test_diag)
print(f'Stacking Classifier Score: {stack_model_score}')

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [02:17:45] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


Stacking Classifier Score: 0.5983392351822356


## Body Part - XGBoost

### Initial Model

In [18]:
X_body = neiss.drop(['CPSC_Case_Number','Other_Race', 'Body_Part','Other_Diagnosis', 'Body_Part_2', 'Diagnosis_2', 'Other_Diagnosis_2', 'PSU', 'Stratum', 'Narrative_1','Treatment_Date'], axis=1)
y_body = le.fit_transform(neiss['Body_Part'])

In [19]:
# Splitting data for `body_part`
X_train_body, X_test_body, y_train_body, y_test_body = train_test_split(X_body, y_body, test_size=0.2, random_state=42)

In [20]:
y_train_body = le.fit_transform(y_train_body)

In [21]:
# Create an XGBClassifier instance with GPU support
model_xg_body = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', tree_method='gpu_hist', gpu_id=0)

In [22]:
# Fit the model on training data
model_xg_body.fit(X_train_body, y_train_body)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning:

[16:50:08] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"




XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, gpu_id=0, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, ...)

In [23]:
# Make predictions
y_pred_body = model_xg_body.predict(X_test_body)
y_pred_body = le.inverse_transform(y_pred_body)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning:

[16:54:03] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"




In [24]:
# Evaluate the model
accuracy_body = accuracy_score(y_test_body, y_pred_body)
print(f'Accuracy of XGBoost Classifier: {accuracy_body}')

Accuracy of XGBoost Classifier: 0.47648796177457514


In [25]:
print(classification_report(y_test_body, y_pred_body))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       859
           1       0.32      0.17      0.22      2394
           2       0.23      0.31      0.27      3718
           3       0.43      0.21      0.28      1478
           4       0.28      0.21      0.24      2102
           5       0.23      0.20      0.21      2047
           6       0.23      0.24      0.23      3296
           7       0.29      0.09      0.14      2322
           8       0.37      0.56      0.45      3206
           9       0.25      0.13      0.17       375
          10       0.89      0.84      0.86     12280
          11       0.41      0.62      0.49      5704
          12       0.44      0.48      0.46       856
          13       0.32      0.46      0.38      4892
          14       0.28      0.09      0.13       703
          15       0.22      0.05      0.08       816
          16       0.34      0.24      0.28      2559
          17       0.37    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [26]:
feature_importances_body = model_xg_body.feature_importances_

In [27]:
feature_names_body = X_train_body.columns

In [28]:
df_sorted_body = pd.DataFrame({'Feature': feature_names_body, 'Importance': feature_importances_body})
df_sorted_body = df_sorted_body.sort_values(by='Importance', ascending=False)

In [29]:
fig = px.bar(df_sorted_body, x='Feature', y='Importance', labels={'x': 'Feature', 'y': 'Importance'},
             title="Feature Importances")
fig.show()

### Feature Importance

In [ ]:
# Drop the irrelevant features
X_body_fi = neiss.drop(['CPSC_Case_Number','Other_Race', 'Body_Part','Other_Diagnosis', 'Body_Part_2', 'Diagnosis_2', 'Other_Diagnosis_2', 'PSU', 'Stratum', 'Narrative_1','Treatment_Date', 'Product_3','Drug', 'Alcohol', 'Fire_Involvement'], axis=1)
y_body_fi = neiss['Body_Part']

In [ ]:
# Splitting data for `body_part`
X_train_body_fi, X_test_body_fi, y_train_body_fi, y_test_body_fi = train_test_split(X_body_fi, y_body_fi, test_size=0.2, random_state=42)

### Best Parameters

In [30]:
best_xgb_model.fit(X_train_body, y_train_body)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning:

[17:29:47] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"




XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=0, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=9, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, ...)

In [31]:
y_pred_body_best = best_xgb_model.predict(X_test_body)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning:

[17:39:59] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"




In [32]:
# Evaluate the model
accuracy_body_best = accuracy_score(y_test_body, y_pred_body_best)
print(f'Accuracy of XGBoost Classifier: {accuracy_body_best}')

Accuracy of XGBoost Classifier: 0.47740029998917566
